# Ontology Practical 

Now we will learn about HPO and semantic similarity.

To run this, you must run this on the command line:

```console
pip3 install --user pyhpo sklearn scikit-plot matplotlib
```

You may have to restart the notebook after you install the things.

Note: ... indicates something you need to fill out yourself. You should be able to find the necessary code earlier in the notebook. Ask if you're having any trouble, or don't understand anything!

In [ ]:
!pip3 install --user pyhpo sklearn scikit-plot matplotlib

Import the necessary files and load the ontology

In [ ]:
from pyhpo.ontology import Ontology
from pyhpo.set import HPOSet
import csv
import sklearn
import sklearn.metrics
import scikitplot as skplt
import matplotlib.pyplot as plt

ontology = Ontology()

We can find and select phenotypes in several different ways

You can use the following box to search for abnormalities, then copy them into the subsequent boxes when required:

In [ ]:
for term in ontology.search('kidney'):
    print(term)

We can use semantic similarity to see how close two terms are. We would expect that two terms that have nothing to do with each other are dissimilar:

In [ ]:
term = ontology.get_hpo_object('Abnormality of mandibular symphysis')
term2 = ontology.get_hpo_object(. . .)

print(term)
print(term2)

term.similarity_score(term2, method = 'resnik')

Can you think of a phenotype that is related to tachycardia? Think about what might influence class similarity.

In [ ]:
terms = [
    ontology.get_hpo_object('Tachycardia'),
    ontology.get_hpo_object(. . .)
]

terms[0].similarity_score(terms[1], method = 'resnik')

Can you think of two other phenotypes that are similar? find them using the methods above, and enter them in here:

In [ ]:
terms = [
    ontology.get_hpo_object(. . .),
    ontology.get_hpo_object(. . .)
]

terms[0].similarity_score(terms[1], method = 'resnik')

We can also identify the similarity between different gruops of phenotypes!. now we can create two patients, by adding a list of phenotypes. select some phenotypes usign the methods above to create the patients, then compare them:

In [ ]:
patient = pyhpo.HPOSet([
    ...
])

patient2 = pyhpo.HPSet([
    ...
])

patient.similarity(patient2)

In the file patients.tsv, we have phenotype characterisations of 100 (simulated) patients with diseases described by OMIM. we can load this and the OMIM database into memory:

In [ ]:
patients = dict()
omim = dict()
diagnosis = dict()
omim_names = dict()

with open('patients.tsv') as tsv:
    reader = csv.reader(tsv, delimiter = '\t')
    for row in reader:
        key = 'patient ' + row[0]
        if not key in patients.keys():
            patients[key] = []
            diagnosis[key] = row[2]
        patients[key].append(row[1])
        
with open('new_phenotype_annotation.tab') as tsv:
    reader = csv.reader(tsv, delimiter = '\t')
    for row in reader:
        key = row[2]
        if not key in omim.keys():
            omim[key] = []
        if not row[2] in omim_names.keys():
            omim_names[row[2]] = row[0]
        omim[key].append(row[1])

Now we can look at some patients, and see what phenotypes they've been characterised with:

In [ ]:
patient_id = 'patient 20'
patient = patients[patient_id]

patient

And we can look at some diseases:

In [ ]:
disease_id = 'OMIM:156620'
disease = omim[disease_id]

print(omim_names[disease_id])
disease

And we can look up what those phenotypes are in HPO ...

In [ ]:
print('patient: ')
for phenotype in patient:
    hpo = ...
    print('  ' + str(hpo))
    
print('disease: ')
for phenotype in disease:
    hpo = ...
    print('  ' + str(hpo))

So it follows that we should be able to diagnosie the patient, by identifying which diseases have all the phenotypes that the patients have. The following function will find diseases that have a particular phenotype:

In [ ]:
def diseasesWithPhenotype(phenotype):
    return [key  for (key, value) in omim.items() if phenotype in value]

So let's see which diseases cause hypertension. Note that we can use omim_names[disease] to look up the name of a disease.

In [ ]:
hypertension = ontology.get_hpo_object('Hypertension').id
for disease in diseasesWithPhenotype(hypertension):
    print(omim_names[disease])

So, for our patient, can we figure out what disease they have, by finding the only disease that contains all the phenotypes?

In [ ]:
patient

Lets start by finding all the diseases that each phenotype can have. Fill in the following:

In [ ]:
diseases_with_phenotypes = [
    set(diseasesWithPhenotype(. . .)),
    . . .
]

Now we can find diseases that have all those phenotypes:

In [ ]:
set.intersection(diseases_with_phenotypes[0], 
                 diseases_with_phenotypes[1], 
                 diseases_with_phenotypes[2], 
                 diseases_with_phenotypes[3],
                 diseases_with_phenotypes[4])

Sadly, it returns nothing. :(. This is because none of the diseases contain all those phenotypes. What could the problem be? Well, lets look at what the patient actually had:

In [ ]:
patient_diagnosis = diagnosis[patient_id]
print(omim_names[patient_diagnosis])
disease_phenotypes = omim[patient_diagnosis]
disease_phenotypes

The first thing you might notice, is that there are quite a lot more phenotypes in the definition of the disease than there are in the patient characterisation.

Why might this be?

Now, let's look at which of our patient's phenotypes are in the disease definition:

In [ ]:
print(patient[0] in disease_phenotypes)
print(patient[1] in disease_phenotypes)
print(patient[2] in disease_phenotypes)
print(patient[3] in disease_phenotypes)
print(patient[4] in disease_phenotypes)

So, you should have discovered that two phenotypes were not included in the disease definition. Why could this be?

Let's look at the semantic similarity between the two phenotypes not included in the disease, and the phenotypes in the disease definition. See if you can enter the function that calculates the similarity score:

In [ ]:
missing_phenotype_one = ontology.get_hpo_object(patient[2])
missing_phenotype_two = ontology.get_hpo_object(patient[4])

print(missing_phenotype_one)
for phenotype in disease_phenotypes:
    phenotype = ontology.get_hpo_object(phenotype)
    score = ...
    print('  Semantic similarity score with ' + phenotype.name + ': ' + str(score))

print(missing_phenotype_two)
for phenotype in disease_phenotypes:
    phenotype = ontology.get_hpo_object(phenotype)
    score = ...
    print('  Semantic similarity score with ' + phenotype.name + ': ' + str(score))


We notice that most of the classes are very dissimilar, but for each of the phenotypes, there is a very similar class in the definition of the disease. 

let's look at the shortest path to that class in the ontology. the first one i have put in there, the second one you can fill yourself

In [ ]:
missing_phenotype_one.path_to_other(ontology.get_hpo_object('Basal ganglia calcification'))

In [ ]:
missing_phenotype_two.path_to_other(...)

So we can see that our missing terms are just more general terms than the ones defined by OMIM. The above shows the path between the two classes. In both cases, it's only 

This makes sense - many patients will not have access to specialists, or the equipment or instruments to make very specific phenotypic profiles of patients.

So the two kinds of phenotyping error we've seen are:

1) Omission
2) Generality

Another kind of error that we haven't seen here (though it exists in this database) is *irrelevant* phenotypes. That is, the recording of a phenotype that a patient does have, but isn't relevant to that particular diagnosis. For example, a patient might have hypertension, but it is not relevant to their diagnosis of a brain injury.

This means that we can't just match patients with the disease profile.

So, can we use semantic similarity to automatically infer what disease a patient has?

You might remember that we're also able to look at semantic similarity between a set of terms. So, let's look at the groupwise semantic similarity between the patient and the disease profile:

In [ ]:
patient_hpos = HPOSet(list(map(ontology.get_hpo_object, patient)))
disease_hpos = HPOSet(list(map(ontology.get_hpo_object, disease_phenotypes)))

patient_hpos.similarity(disease_hpos)

So, there is some similarity, but without context it doesn't really mean anything. lets try comparing it with every disease in OMIM:

In [ ]:
similarities = dict()
for d in omim:
    name = ...
    phens = list(map(ontology.get_hpo_object, omim[d]))
    score = patient_hpos.similarity(phens)
    similarities[d] = score
    print(name + ': ' + str(score))

Well, there are quite a lot, so let's get the top ten:

In [ ]:
import operator
sorted_similarities = sorted(similarities.items(), key=operator.itemgetter(1), reverse = True)
for i in range(0, 30):
    print(omim_names[sorted_similarities[i][0]] + ': ' + str(sorted_similarities[i][1]))

And we can see that, indeed, the most similar disease to our patient's phenotypic profile is the disease that the patient has:

In [ ]:
omim_names[patient_diagnosis] == omim_names[sorted_similarities[0][0]]

Now try to select your own patient, and use components of the code above to perform your own semantic similarity experiment. See if you can find a patient for whom semantic similarity fails to find the right disease. 

In [ ]:
patient_id = . . .
patient = patient[patient_id]
patient_diagnosis = diagnosis[patient_id]

. . .

Depending on the level of inspecificity, we may not always be correct. However, this should still be useful as a hypothesis finding tool.

We can measure how well we find the correct disease, by generating the similarities. By creating a score for each disease for each patient, we test how good the semantic similarity classifier is at *ranking* the 

This will probably take some time, since there is a lot of processing. If you are a computer scientist, maybe you can make it multiprocess.

In [ ]:
scores = list()
truth = list()

for p in patients:
    patient_hpos = HPOSet(list(map(ontology.get_hpo_object, patients[p])))
    for d in omim:
        name = omim_names[d]
        phens = HPOSet(list(map(ontology.get_hpo_object, omim[d])))
        score = patient_hpos.similarity(phens)
        does_patient_have_disease = diagnosis[p] == ...
        
        if does_patient_have_disease:
            print(diagnosis[p] + ' ' + d + ' ' + str(does_patient_have_disease))

        scores.append(score)
        truth.append(does_patient_have_disease)
        
print('done')


In science, it is common to have to wait. But normally, after waiting, it won't work and then you'll have to do it all again. Hopefully, in this case, it's fine, we can finally calculate an AUC:

In [ ]:
auc = sklearn.metrics.roc_auc_score(truth, scores)
fpr, tpr, _ = sklearn.metrics.roc_curve(truth, scores)
plt.plot(fpr, tpr, label="OMIM disease prediction, AUC= " + str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
for term in ontology.search('hypertension'):
    print(term)